In [ ]:
#COMS 3007 NB Lab
    # 
    # Takatso Molekane 569869
    # Nosipho Nkosi 1080888
    #
    #

In [2]:
from sklearn import datasets
from sklearn import metrics
import numpy as np
import pandas as pd
import math
import random
from sklearn.naive_bayes import GaussianNB

In [3]:
feature_names = np.array(['word', 'negative', 'positive']) # List of feature names

In [4]:
# Reading in the data points
data_file = open('simple-food-reviews.txt', 'r')
data_string = data_file.read()
full_data = np.array(data_string.split('\n'))
np.random.shuffle(full_data)
training_data = full_data[0:12]
test_data = full_data[13:19]

In [5]:
def word_count(data):
    training_data = ""
    for i in range(len(data)):
        training_data += data[i]+ " "
    
    training_data = training_data.split()
    arr, counts = np.unique(training_data, return_counts = True)
    return (arr, counts)

word_count(training_data)


(array(['-1', '1', 'a', 'and', 'at', 'avoid', 'bad', 'enjoyed',
        'experience', 'food', 'great', 'had', 'i', 'is', 'lovely', 'meal',
        'my', 'really', 'restaurant', 'service', 'tasted', 'terrible',
        'the', 'this', 'was', 'we', 'what'], dtype='<U10'),
 array([7, 5, 4, 1, 1, 2, 2, 1, 1, 4, 2, 1, 1, 3, 2, 2, 1, 1, 5, 2, 1, 3,
        8, 3, 3, 1, 1]))

In [15]:
#Calculate Priors
def priors(data):
    words, total_wc = word_count(data)
    words = words[2:]
    total_wc = total_wc[2:]
    
    neg = "-"
    negative_review = " "
    positive_review = " "
    p_c = 0
    n_c = 0
    wc = np.zeros(words.shape[0])
    
    #From the training data we want to seperate neg and pos rev's
    i = 0
    while i < len(data):
        if data[i][0] == neg:
            negative_review += data[i] + " "
            i += 1
            n_c += 1
        else:
            positive_review += data[i] + " "
            i += 1
            p_c += 1

    #Overall Probabilities
    prob_neg_rev = n_c/len(data)
    prob_pos_rev = p_c/len(data)

    nr, nr_c = np.unique(negative_review.split(), return_counts=True)
    pr, pr_c = np.unique(positive_review.split(), return_counts=True)

    #We want to ignore '-1' and '1'
    nr = nr[1:]
    nr_c = nr_c[1:]
    pr = pr[1:]
    pr_c = pr_c[1:]
    
    prob_neg = np.zeros(words.shape[0])
    prob_pos = np.zeros(words.shape[0])
    
    for i in range(len(words)):
        for j in range(len(nr)):
            if words[i] == nr[j]:
                prob_neg[i] = nr_c[j]/total_wc[i]
    
    for i in range(len(words)):
        for j in range(len(pr)):
            if words[i] == pr[j]:
                prob_pos[i] = pr_c[j]/total_wc[i]
     
    return prob_neg_rev, prob_pos_rev, prob_pos, prob_neg

priors(training_data)  

(0.5833333333333334,
 0.4166666666666667,
 array([0.75      , 0.        , 1.        , 0.        , 0.        ,
        1.        , 1.        , 0.25      , 1.        , 1.        ,
        1.        , 0.33333333, 1.        , 0.5       , 1.        ,
        0.        , 0.6       , 0.        , 1.        , 0.        ,
        0.25      , 0.33333333, 0.        , 1.        , 1.        ]),
 array([0.25      , 1.        , 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.75      , 0.        , 0.        ,
        0.        , 0.66666667, 0.        , 0.5       , 0.        ,
        1.        , 0.4       , 1.        , 0.        , 1.        ,
        0.75      , 0.66666667, 1.        , 0.        , 0.        ]))